In [1]:
import pandas as pd
import numpy as np
from github import Auth
from github import Github
import json
import pickle
import os
import time
import shutil
from tqdm import tqdm

In [ ]:
ACCESS_TOKEN = json.load(open("./config"))["access_token"]

REPO_METADATA_PATH = "./Data/BlockchainAppRepositories-New.csv"

REPOSITORIES_BIN_DATA_PATH = "./Data/Repositories.bin"
ISSUE_DATA_PATH = "./Data/AllIssues-New.csv"

BACKUP_PATH = "./Data/Backup"

In [3]:
auth = Auth.Token(ACCESS_TOKEN)
github = Github(auth=auth)

In [4]:
def rate_limited(interval=600):
    last_executed = 0 

    def decorator(func):
        def wrapper(*args, **kwargs):
            nonlocal last_executed
            current_time = time.time()
            if current_time - last_executed >= interval:
                last_executed = current_time
                return func(*args, **kwargs)
            else:
                pass
        return wrapper
    return decorator

@rate_limited(120)
def backup(paths):
    if not os.path.isdir(BACKUP_PATH):
        os.mkdir(BACKUP_PATH)
    for path in paths:
        if os.path.exists(path):
            shutil.copyfile(path, BACKUP_PATH + "/" + os.path.basename(path) + time.strftime("%Y%m%d-%H%M%S"))

class Repository:
    def __init__(self, github_client, repo_url):
        self.github_client = github_client
        self.identifier = self.parse_repo_identifier(repo_url)
        self.issues = []
        self.issue_comments = {}

    def parse_repo_identifier(self, repo_url):
        owner = repo_url.split("/")[-2]
        repo_name = repo_url.split("/")[-1]
    
        return owner + "/" + repo_name
    
    def get_identifier(self):
        return self.identifier

    def collect_info(self):
        self.repo = self.github_client.get_repo(self.identifier)
        self.set_issues(self.repo.get_issues(state="all"))

    def set_issues(self, issues):
        self.issues = issues
        for issue in self.issues:
            self.set_comments(issue.number, issue.get_comments())

    def set_comments(self, issue_id, comments):
        self.issue_comments[issue_id] = comments

    def get_issues(self):
        issues = []
        for issue in self.issues:
            issues.append({
                "Repository": self.identifier,
                "IssueId": issue.number,
                "Title": issue.title,
                "Body": issue.body,
                "State": issue.state,
                "Label": ",".join([label.name for label in issue.labels]),
                "CreatedAt": issue.created_at,
                "ClosedAt": issue.closed_at
            })
        return pd.DataFrame(issues)

    ## Not necessary and takes too much time
    def get_comments(self):
        comments = []
        for issue_id in self.issue_comments.keys():
            for comment in self.issue_comments[issue_id]:
                comments.append({
                    "Repository": self.identifier,
                    "IssueId": issue_id,
                    "CommentId": comment.id,
                    "Comment": comment.body
                })
        return pd.DataFrame(comments)

In [ ]:
repositories = []

processed_repos = []
if os.path.exists(ISSUE_DATA_PATH):
    processed_repos = pd.read_csv(ISSUE_DATA_PATH)["Repository"].unique().tolist()

df = pd.read_csv(REPO_METADATA_PATH)

for url in tqdm(df["repo_url"].to_list()):
    try:
        repo = Repository(github, url)
        if not repo.get_identifier() in processed_repos:
            repo.collect_info()
            repositories.append(repo)

            if os.path.exists(ISSUE_DATA_PATH):
                issues = pd.read_csv(ISSUE_DATA_PATH)
                issues = pd.concat([issues, repo.get_issues()], ignore_index=True, sort=False)
            else:
                issues = repo.get_issues()
            
            backup([ISSUE_DATA_PATH, REPOSITORIES_BIN_DATA_PATH])

            issues.to_csv(ISSUE_DATA_PATH, index=False)
            #pickle.dump(repositories,  open(REPOSITORIES_BIN_DATA_PATH, 'wb'))
    except Exception as e:
        print("Error: ", e)

  0%|          | 1/1194 [01:05<21:37:35, 65.26s/it]

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}


  2%|▏         | 25/1194 [27:28<11:04:50, 34.12s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  2%|▏         | 26/1194 [27:44<9:16:28, 28.59s/it] <ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  2%|▏         | 27/1194 [27:57<7:43:54, 23.85s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  2%|▏         | 28/1194 [28:24<8:02:39, 24.84s/it]Following Github server redirection from /repos/Sparklemobile/Sparkle-Token to /repositories/161784150
<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DA

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}


<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  8%|▊         | 96/1194 [1:15:25<10:31:51, 34.53s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  8%|▊         | 97/1194 [1:16:07<11:05:58, 36.43s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  8%|▊         | 98/1194 [1:17:21<14:04:50, 46.25s/it]Following Github server redirection from /repos/jpmorganchase/anonymous-zether to /repositories/186751312
<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
  8%|▊         | 99/1194 [1:17:5

...

<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 96%|█████████▌| 1146/1194 [14:23:19<27:06, 33.89s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 96%|█████████▌| 1147/1194 [14:24:16<32:09, 41.05s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 96%|█████████▌| 1148/1194 [14:25:10<34:15, 44.68s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 96%|█████████▌| 1149/1194 [14:26:01<35:00, 46.68s/it]Following Github server redirection from /repos/harry1453/burstcoin-explorer-android

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}


<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 97%|█████████▋| 1160/1194 [14:35:10<24:26, 43.14s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 97%|█████████▋| 1161/1194 [14:36:01<25:00, 45.48s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 97%|█████████▋| 1162/1194 [14:36:57<25:55, 48.61s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 97%|█████████▋| 1163/1194 [14:37:54<26:24, 51.12s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Speci

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository", "status": "404"}


<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 98%|█████████▊| 1170/1194 [14:43:19<16:59, 42.50s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 98%|█████████▊| 1171/1194 [14:44:12<17:31, 45.71s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 98%|█████████▊| 1172/1194 [14:45:07<17:50, 48.68s/it]<ipython-input-5-6a6abf91fc04>:17: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv(ISSUE_DATA_PATH)
 98%|█████████▊| 1173/1194 [14:46:14<18:53, 53.98s/it]Following Github server redirection from /repos/flamingYawn/ethereum-block-explorer 